# Projecto: BookFeelsVerse

## 1. Extracción de Libros Más Vendidos desde Crisol

In [1]:
# Librerias utilizadas
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

# URL de la página
url = "https://www.crisol.com.pe/otros/los-mas-vendidoss"

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de respuesta 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    titulos = []
    autores = []
    precios = []
    ofertas = []

    # Encontrar todos los elementos div que contienen información de los libros
    libros = soup.find_all('div', class_='product-item-info')

    # Iterar sobre los elementos y extraer la información
    for libro in libros:
        # Obtener Título
        titulo = libro.find('strong', class_='product name product-item-name')
        titulo_text = titulo.text.strip() if titulo else 'No Title'
        titulos.append(titulo_text)

        # Obtener Precios y Ofertas
        for price_box in libro.find_all('div', class_='price-box'):
            special_price = price_box.find('span', class_='special-price')
            old_price = price_box.find('span', class_='old-price')

            # Extraccion de los precios
            if old_price:
                precio_elem = old_price.find('span', class_='price').text.strip()
            else:
                precio_elem = price_box.find('span', class_='price').text.strip()

            # Extracción de las ofertas
            oferta_elem = special_price.find('span', class_='price').text.strip() if special_price else '-'

            # Append to lists
            precios.append(precio_elem)
            ofertas.append(oferta_elem)

        # Obtener Autores
        autor_elems = libro.find_all('div', class_='author')
        autores_text = ', '.join(set([autor.text.strip() for autor in autor_elems]))
        autores.append(autores_text)

    # Crear un DataFrame con los datos
    df = pd.DataFrame({
        'Titulo': titulos,
        'Autores': autores,
        'Precio': precios,
        'Oferta': ofertas
    })

# Imprimir el DataFrame resultante
df

,Titulo,Autores,Precio,Oferta
0,Cuando no queden más estrellas ...,"Martínez, María",S/ 59.90,-
1,Perronejo,"Ramos, Wendy",S/ 69.00,S/ 51.75
2,Tea Shop,"Pinasco, Bruno",S/ 79.90,S/ 59.93
3,Los genios,"Bayly, Jaime",S/ 79.00,S/ 59.25
4,Hábitos atómicos,"Clear, James",S/ 99.90,S/ 74.93
5,Le dedico mi silencio,"Vargas Llosa, Mario",S/ 89.00,S/ 66.75
6,Cómo Hacer que Te Pasen Cosas B...,"Rojas Estapé, Marian",S/ 89.90,S/ 67.43
7,La teoría de Kim,"Sandoval, Jay",S/ 99.00,S/ 74.25
8,Cien cuyes,"Rodríguez, Gustavo",S/ 69.00,S/ 51.75
9,"Rojo, blanco y sangre azul","McQuiston, Casey",S/ 89.00,S/ 66.75


## 2. Extracción de Información de Libros en Goodreads

In [2]:
# Importar las librerías necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Lista de IDs de libros en Goodreads
ids = ['58295801-cuando-no-queden-m-s-estrellas-que-contar', '202329456-perronejo', '200276684-tea-shop',
       '122800780-los-genios', '45028173-h-bitos-at-micos', '192888751-le-dedico-mi-silencio',
       '41811331-c-mo-hacer-que-te-pasen-cosas-buenas', '196963397-la-teor-a-de-kim', '91819656-cien-cuyes',
       '52950904-rojo-blanco-y-sangre-azul', '38913520-el-sutil-arte-de-que-te-importe-un-caraj', '141869624-la-teor-a-del-amor',
       '42744102-el-club-de-las-5-de-la-ma-ana', '193543529-presidentes-por-accidente', '54659294-tres-meses']

# URL base para los libros en Goodreads
base_url = 'https://www.goodreads.com/book/show/'
urls = [base_url + book_id for book_id in ids]

# Listas para almacenar la información extraída
titulos = []
puntuaciones = []
todas_reseñas = []

# Iterar a través de las URLs de los libros
for url in urls:
    # Realizar la solicitud HTTP para obtener la página
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Utilizar BeautifulSoup para analizar el contenido HTML de la página
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extraer el título del libro
        title_elem = soup.find('h1', {'class': 'Text Text__title1'})
        title = title_elem.text.strip() if title_elem else None
        titulos.append(title)

        # Extraer la puntuación del libro
        rating_elem = soup.find('div', class_='RatingStatistics__rating')
        rating = rating_elem.text.strip() if rating_elem else None
        puntuaciones.append(rating)

        # Extracción de las reseñas (cambiar según la estructura de la página)
        reseñas = []
        for reviews in soup.find_all('div', class_='ReviewsList'):
            reviews_elems = reviews.find_all('span', class_='Formatted')
            for review_elem in reviews_elems:
                review_text = review_elem.text.strip()
                reseñas.append(review_text)

        # Almacenar las primeras 5 reseñas en la lista principal
        todas_reseñas.append(reseñas)
    else:
        # Imprimir un mensaje de error si la solicitud no fue exitosa
        print(f'Error al obtener la página {url}. Código de estado: {response.status_code}')

# Crear un DataFrame con la información recopilada
df1 = pd.DataFrame({'Título': titulos, 'Puntuación': puntuaciones, 'Reseñas': todas_reseñas})

# Imprimir el DataFrame resultante
df1

,Título,Puntuación,Reseñas
0,Cuando no queden más estrellas que contar,4.43,[ME HA FASCINADO. No voy a cansarme de recomen...
1,Perronejo,4.78,[5 stars. A little ball of fluff in the desert...
2,Tea Shop,3.14,[De verdad quise que este libro me guste pero ...
3,Los genios,4.00,"[""Esto es por lo que NO le hiciste a Patricia""..."
4,Hábitos atómicos,4.36,[A Book That Changed My LifeA book worth readi...
5,Le dedico mi silencio,3.62,"[""Nada mas huachafo que llorar en la entrega d..."
6,Cómo hacer que te pasen cosas buenas,4.13,[Lo peor de este libro es el título.A pesar de...
7,La teoría de Kim,4.68,[2.5 WAR IS FINALLY OVER.genuinamente puedo de...
8,Cien cuyes,4.14,"[Últimamente, pocos libros me conmueven tanto ..."
9,"Rojo, blanco y sangre azul",4.10,[I am a soulless void. I have definitely said ...
